In [2]:
%load_ext autoreload
%autoreload 1

In [3]:
from scipy.stats import randint, uniform
from sklearn.model_selection import train_test_split
from src.data_loader import *
from sklearn.preprocessing import label_binarize, PowerTransformer
from PIL import Image
from IPython.display import Image, display
from src.pipeline import *
import joblib
from src.convert_graphml_to_pyg_data_multithread import *
from tqdm import tqdm

import networkx as nx
%aimport src.data_loader
%aimport src.visualization
%aimport src.training
%aimport src.evaluation
%aimport src.pipeline
%aimport src.convert_graphml_to_pyg_data_multithread

In [4]:
features = [
            "faces", "edges", "vertices", "quantity",
            "height", "width", "depth", "volume", "area",
            "bbox_height", "bbox_width", "bbox_depth", "bbox_volume",
            "bbox_area",
        ]
data = pd.read_csv("./data/synced_dataset_final.csv")

In [5]:
X = data[features]
X_train, X_test, y_train_index, y_test_index = train_test_split(
    X, range(len(X)),
    test_size=0.2,
    random_state=100,
    stratify=data["is_cnc"])
y_multi_train = data["multiclass_labels"].iloc[y_train_index]
y_multi_test = data["multiclass_labels"].iloc[y_test_index]

y_binary_train = data["is_cnc"].iloc[y_train_index]
y_binary_test = data["is_cnc"].iloc[y_test_index]


params = {
    "n_estimators": randint(100, 300),
    "max_depth": randint(3, 10),
    "learning_rate": uniform(0.01, 0.3),
}

In [6]:
best_search_binary = randomizedsearchcv_xgboost(X_train, y_binary_train, params)

In [7]:
y_binary_pred = best_search_binary.best_estimator_.predict(X_test)
y_binary_prob = best_search_binary.best_estimator_.predict_proba(X_test)

In [8]:
print("Binary Model Evaluation")
metrics_binary = evaluate_classification(
    y_true=y_binary_test,
    y_pred=y_binary_pred,
    y_prob=y_binary_prob,
)
pprint(metrics_binary)

Binary Model Evaluation
{'accuracy': 0.8711414790996784,
 'average_precision': 0.9604947229991485,
 'f1_macro': 0.8609816827884358,
 'f1_weighted': 0.8718514262103362,
 'roc_auc': 0.9350805013844213}


In [9]:
y_binary_pred_train = best_search_binary.best_estimator_.predict(X_train)
y_binary_prob_train = best_search_binary.best_estimator_.predict_proba(X_train)

metrics_binary_train = evaluate_classification(
    y_true=y_binary_train,
    y_pred=y_binary_pred_train,
    y_prob=y_binary_prob_train,
)
pprint(metrics_binary_train)

{'accuracy': 0.9465814542385144,
 'average_precision': 0.9942039999295788,
 'f1_macro': 0.9427195107495587,
 'f1_weighted': 0.9470214527536648,
 'roc_auc': 0.9892845813353901}
